# RandomForestによる教師あり学習

---
## 目的
RandomForestを用いて2つのサンプルの識別を行う．その後，交差検定法を用いて識別テストを行う．


## プログラムの動作
`08_RandomForest.py`を実行すると，`data/car.txt`と`data/human.txt`の2つ読み込む．次に，RandomForestによる学習およびテストを交差検定法を用いて行う．最後に，識別率と識別されたグラフを表示する．


## プログラムの解説
プログラムを上から順番に解説していく．なお，今回も前回，前々回と同じく**第5回と同じプログラムで識別器の定義とモジュールのインポートを変更しただけ**であるため，細かい説明は省略する．わからないところがあれば第5回の解説を読み返すこと．



## 準備
プログラムの動作に必要なデータをダウンロードし，zipファイルを解凍する．

In [ ]:
!wget -q http://www.mprg.cs.chubu.ac.jp/tutorial/ML_Lecture/sklearn/data.zip
!unzip -q data.zip
!ls
!ls ./data

## モジュールのインポート
初めに，必要なモジュールをインポートする．

今回は新たに`RandomForestClassifier`をインポートする．

In [ ]:
from os import path
import numpy as np
from sklearn import metrics
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

##データの読み込み
次に，テキストファイルを読み込む．

In [ ]:
in_txt1 = open(path.join('data', 'car.txt'))
in_txt2 = open(path.join('data', 'human.txt'))

car = np.asarray([(line.strip()).split('\t') for line in in_txt1], dtype=float)
print(car.shape)
human = np.asarray([(line.strip()).split('\t') for line in in_txt2], dtype=float)
print(human.shape)

## データのラベル付けと結合，交差検定法の準備
学習を行う前に，データがcarまたはhumanどちらのクラスに属するかラベル付けをする．

In [ ]:
car_y =  np.zeros(car.shape[0])
human_y = np.ones(human.shape[0])
X= np.r_[car, human]
y= np.r_[car_y, human_y]

kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=0)
print(X.shape, y.shape)

## 学習と評価
for文で学習と評価をk回繰り返す．

In [ ]:
scores = []
best_score = 0.0

for train, test in kfold.split(X):
    classifier = RandomForestClassifier(n_estimators=100, max_depth=3, criterion="entropy")
    classifier.fit(X[train], y[train])
    preds = classifier.predict(X[test])
    score = metrics.accuracy_score(preds, y[test])
    scores.append(score)

    if score > best_score:
        best_classifier = classifier
        best_score = score

`classifier = RandomForestClassifier(n_estimators=100, max_depth=3, criterion="entropy")`が識別器の定義となる．主なパラメータとして`n_estimators`と`max_depth`と`criterion`がある．

- `n_estimators`: 決定木の数
- `max_depth`: 決定木の最大の深さ
- `criterion`: 分割の品質を計算する関数


## 識別率の計算
得られたk個の結果を平均して，識別率を求める．

In [ ]:
accuracy = (sum(scores) / len(scores)) * 100
msg = 'recognition rate: {accuracy:.2f}%'.format(accuracy=accuracy)
print(msg)

## グラフの描画
識別結果を可視化するためにグラフを作成する．どちらのクラスがどの領域かわかりやすく表示することができる．

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
subfig = fig.add_subplot(1,1,1)
plt.xlim(0, 10000)
plt.ylim(20, 50)

xx, yy = np.meshgrid(np.linspace(plt.xlim()[0], plt.xlim()[1], 500),
                     np.linspace(plt.ylim()[0], plt.ylim()[1], 500))

Z = best_classifier.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
cs = plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)

subfig.scatter(car[:,0], car[:,1],color='black')
subfig.scatter(human[:,0], human[:,1],color='red')

subfig.set_title('08 RandomForest')
subfig.set_xlabel('Area')
subfig.set_ylabel('complexity')

plt.savefig("08_graph.png")
plt.show()

## 課題
1. 木の数（`n_estimators`）と深さ（`max_depth`）を変更すると識別境界はどうなるか．グラフで確認せよ．
2. 識別率の最も高くなるパラメータを求めよ．


## ヒント
1. 値を増やしたり減らしたりして，それぞれどう変わるか確認してみる．
2. 決定木の深さが深すぎると過学習になる．